In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


from readable_implementations.utils import Trainer
from readable_implementations.utils.config import BasicConfig
from readable_implementations.models import VisionTransformer

In [20]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [25]:

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

config = BasicConfig(
    device = "cpu",
    model = net,
    optimizer=optimizer,
    loss_fn=criterion,
    max_epochs=2,
    train_dataloader=trainloader,
    val_dataloader=testloader,
    validate_every_batch= 6000,
)

trainer = Trainer(config)

In [26]:
trainer.fit()

INFO:train:E:1 (4%), step=500, loss=0.575                     
INFO:train:E:1 (8%), step=1000, loss=0.566                    
INFO:train:E:1 (12%), step=1500, loss=0.538                    
INFO:train:E:1 (16%), step=2000, loss=0.513                    
INFO:train:E:1 (20%), step=2500, loss=0.509                    
INFO:train:E:1 (24%), step=3000, loss=0.483                    
INFO:train:E:1 (28%), step=3500, loss=0.476                    
INFO:train:E:1 (32%), step=4000, loss=0.459                    
INFO:train:E:1 (36%), step=4500, loss=0.439                    
INFO:train:E:1 (40%), step=5000, loss=0.426                    
INFO:train:E:1 (44%), step=5500, loss=0.425                    
validation: 100%|██████████| 2500/2500 [00:05<00:00, 477.70it/s]
INFO:train:Validation at 24000 step: acc: 0.39 , f1-macro: 0.38 , f1-micro: 0.38 , val_loss: 1.67
INFO:train:E:1 (48%), step=6000, loss=0.422                    
INFO:train:E:1 (52%), step=6500, loss=0.407                    
INFO:tr